# A)

In [1]:
# bring here the Value class code and update accordly, hint: follow other operations and loss function code

In [1]:
import math


class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out


  # POW SOLUTION:

    def __pow__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data ** other.data, (self, other), f'**{other}')

        def _backward():
            self.grad += (other.data * self.data**(other-1).data) * out.grad
            # update other gradient
            other.grad += ((self.data ** other.data) * math.log(other.data)) * out.grad
        out._backward = _backward

        return out


  # LOG SOLUTION:

    def log(self):
        # define out
        # define backward function
        # hint: see into another functions here...
        out = Value(math.log(self.data), (self,), 'ln')

        def _backward():
            self.grad += (1 / self.data) * out.grad
        out._backward = _backward

        return out


    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

In [2]:
a=Value(2,_op="a")
b=Value(3,_op="b")

e=Value(math.e, _op="e")

In [3]:
c=a**b
d = c + e
f = d.log()

In [4]:
print(f)

Value(data=2.3719508656009087, grad=0)


In [5]:
from math import log
import numpy as np

f.backward()

print(f"a grad: {a.grad}")
print(f"b grad: {b.grad}")
print(f"c grad: {c.grad}")
print(f"e grad: {e.grad}")

a grad: 1.1195824286069578
b grad: 0.8199913428296602
c grad: 0.09329853571724647
e grad: 0.09329853571724647


# B)

In [9]:
from diabetes import prepare_data
import torch
import random
import numpy as np
import math
import nn

In [10]:
# uncomment to get data

X_train, X_test, y_train, y_test = prepare_data()

In [11]:
# print the shape of the data to get a sense of what is in there
# you should have around 300 samples for train, and 100 for test...
# similarly, around 8 features, and 1 target.


X_train.shape, X_test.shape, y_train.shape, y_test.shape

((331, 10), (111, 10), (331,), (111,))

In [14]:
# initialize a model (using components from nn.py)
# insert correct arguments

model = nn.MLP(10, [20, 1])

print(model)                            
print("number of parameters", len(model.parameters()))

MLP of [Layer of [ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10), ReLUNeuron(10)], Layer of [LinearNeuron(20)]]
number of parameters 241


In [15]:
def get_dataloaders(x, y, batch_size=None, seed=0):

    if batch_size is None:   # all data in a single batch
      xb, yb = x, y 
    else:   # mini-batch
      np.random.seed(seed)                 # randomly shuffle the data 
      ri = np.random.permutation(x.shape[0])
      xb, yb = x[ri], y[ri]
      
      
      total_batches = math.floor(xb.shape[0] / batch_size) # the total batches with given sample size
      total_samples = total_batches * batch_size  # the total samples with sample size
      xb = np.split(xb[:total_samples], total_batches, 0)  # we split by batches with the exact samples (indexing)
      yb = np.split(yb[:total_samples], total_batches, 0)
      xb, yb = np.array(xb), np.array(yb)  # we make back to an array

      print(f'x with shape of {xb.shape}')
      print(f'y with shape of {yb.shape}')

      # we prepare the data in list of Values as required by micro-grad
      xinputs = []
      for x in xb:

        xinput = [list(map(Value, xrow)) for xrow in x]
        xinputs.append(xinput)

      return xinputs, yb

In [18]:
x_train_loader, y_train_loader = get_dataloaders(X_train, y_train, batch_size=16)

x with shape of (20, 16, 10)
y with shape of (20, 16)


In [19]:
learning_rate = 1e-3
epochs = 10
print_loss_every_n_iteration = 1 
total_iterations = epochs * len(x_train_loader)

In [35]:
# training loop 
iteration_number = 0
for epoch in range(epochs):      # for each epoch
  for xtrain, ytrain in zip(x_train_loader, y_train_loader):  # for each iteration
    scores = list(map(model, xtrain))   # we fit a batch into the model and get scores

    # compute the MSE for all samples
    losses = [(score - yt)**2 for score, yt in zip(scores, ytrain)]
    
    # get the mean of all the sample losses for the iteration here (using operations supported by the value class)
    iteration_loss = sum(losses) / len(losses)
    
    model.zero_grad()
    iteration_loss.backward()
    for parameter in model.parameters():
      parameter.data -= learning_rate * parameter.grad 
    iteration_number += 1 
    if iteration_number % print_loss_every_n_iteration == 0:
      print(f'iteration {iteration_number} of {total_iterations} with a loss: {iteration_loss.data}')

iteration 1 of 200 with a loss: 4294.530047312885
iteration 2 of 200 with a loss: 5491.87322445898
iteration 3 of 200 with a loss: 5103.273780020547
iteration 4 of 200 with a loss: 6431.8943905617225
iteration 5 of 200 with a loss: 3735.5899426852898
iteration 6 of 200 with a loss: 5172.572700909284
iteration 7 of 200 with a loss: 4465.56706356632
iteration 8 of 200 with a loss: 5389.846677192501
iteration 9 of 200 with a loss: 3519.030169301523
iteration 10 of 200 with a loss: 4073.8577015071232
iteration 11 of 200 with a loss: 4667.336794469692
iteration 12 of 200 with a loss: 3447.8526290952973
iteration 13 of 200 with a loss: 7353.914698414849
iteration 14 of 200 with a loss: 6593.182679043102
iteration 15 of 200 with a loss: 6740.87367111995
iteration 16 of 200 with a loss: 4697.336830730052
iteration 17 of 200 with a loss: 3122.65552722587
iteration 18 of 200 with a loss: 3331.7234886880096
iteration 19 of 200 with a loss: 8608.475943738005
iteration 20 of 200 with a loss: 3682.0

In [39]:
# let's test

scores = list(map(model, X_test))

# remember to update as per MSE 
results = [(score - yt)**2 for score, yt in zip(scores, ytrain)]
results = [value.data for value in results]

# look at your result and compare with the statistics below
np.mean(results)

5216.971418367859

In [40]:
y_train_loader.min(), y_train_loader.max(), y_train_loader.mean(), y_train_loader.std()

(25.0, 346.0, 154.09375, 77.63848891456801)